In [ ]:
import pickle

# Unpickle the file
with open(r'/content/images.p', "rb") as f:
  images = pickle.load(f)

with open(r'/content/labels.p', "rb") as f:
  labels = pickle.load(f)

In [ ]:
print(images.shape)
print(labels.shape)

In [ ]:
set(labels)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
labels = le.fit_transform(labels)

In [ ]:
set(labels)

In [ ]:
le.inverse_transform([0, 1, 2, 3, 4, 5, 6, 7, 8])

In [ ]:
import numpy as np

In [ ]:
n_persons = len(set(labels))
print("Number of persons: ", n_persons)
label_mapping = le.inverse_transform(np.arange(n_persons))
for i in range(len(label_mapping)):
  print(i, "-->", label_mapping[i])

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(images[200], cmap=plt.get_cmap("gray"))
plt.show()

In [ ]:
import cv2

In [ ]:
def preprocessing(img):
  img = cv2.equalizeHist(img)
  img = img.reshape(100, 100, 1)
  img = img/255
  return img

In [ ]:
images = np.array(list(map(preprocessing, images)))
print("Shape of Input: ", images.shape)

In [ ]:
print(images)

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
labels = to_categorical(labels)
print(labels)

In [ ]:
categories = labels.shape[1]
print(categories)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
# import convolution layer
from tensorflow.keras.layers import Conv2D

# import pooling layer
from tensorflow.keras.layers import MaxPooling2D

# import faltten layer
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import Adam

In [ ]:
def modified_model():

  model=Sequential()

  model.add(Conv2D(64,(3,3),input_shape=(100,100,1),activation='relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))

  model.add(Conv2D(64,(3,3),activation='relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))

  model.add(Conv2D(128,(3,3),activation='relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))

  model.add(Flatten())

  model.add(Dense(800,activation='relu'))

  model.add(Dense(700,activation='relu'))

  model.add(Dense(600,activation='relu'))

  model.add(Dense(500,activation='relu'))

  model.add(Dense(9,activation='softmax'))
  model.compile(Adam(lr=0.002),loss='categorical_crossentropy',metrics=['accuracy'])
  return model

In [ ]:
model=modified_model()
print(model.summary())

In [ ]:
h=model.fit(images,labels,batch_size=80,epochs=40,validation_data=(images,labels),shuffle='True',verbose=1)

In [ ]:
plt.plot(h.history['accuracy'])
plt.plot(h.history['val_accuracy'])
plt.legend(['accuracy','val_accuracy'])
plt.show()

In [ ]:
plt.plot(h.history['loss'])
plt.plot(h.history['val_loss'])
plt.legend(['loss','val_loss'])
plt.show()

In [ ]:
from tensorflow.keras.models import Model

In [ ]:
layer0 = Model(model.layers[0].input, model.layers[0].output)
features = layer0.predict(images[200].reshape(1,100,100,1))

In [ ]:
features.shape

In [ ]:
plt.figure(figsize=(10,15))
for i in range(32):
  axes = plt.subplot(8, 4, i+1)
  plt.imshow(features[0,:,:,i])

In [ ]:
from google.colab import files
upload=files.upload()

In [ ]:
d=list(upload.keys())[0]

In [ ]:
import cv2
a=np.fromstring(upload[d],np.uint8)
img=cv2.imdecode(a,cv2.IMREAD_COLOR)
img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
print(a)
plt.imshow(img,cmap=plt.get_cmap("gray"))

In [ ]:
img=np.asarray(img)
img=cv2.resize(img,(100,100))
img=preprocessing(img)



In [ ]:
img=img.reshape(1,100,100,1)
print(model.predict(img))

In [ ]:
prediction=model.predict(img)

In [ ]:
p=np.argmax(prediction,axis=1)
p

In [ ]:
model.save("final_model.h5")